# GAN

In [1]:
import numpy as np

In [3]:
def discriminator(x1, x2, theta_1, theta_2):
    return(1/(1+np.exp(-(theta_1 * x1 - theta_2*x2 -2))))
def uniform_proba(X):
    return 1 / len(X)

<img src="img/VDG.png"/>

In [4]:
def compute_VDG(X_real, X_fake, theta_1, theta_2):
    # compute the expectation of pdata(X)[lnD(X)]
    proba_X_real = uniform_proba(X_real)
    print('Proba X_real:', proba_X_real)
    Ex_pdata_log_Dx = 0
    for i, x in enumerate(X_real):
        x1, x2 = x
        Ex_pdata_log_Dx += proba_X_real * np.log(discriminator(x1, x2, theta_1, theta_2))
    print('Ex_pdata_log_Dx:', Ex_pdata_log_Dx)
    # compute the expectation of pz(Z)[ln(1-D(G(z)))]
    proba_X_fake = uniform_proba(X_fake)
    Ex_pz_log_1_minus_D_Gz = 0
    for i, x in enumerate(X_fake):
        x1, x2 = x
        Ex_pz_log_1_minus_D_Gz += proba_X_fake * np.log(1 - discriminator(x1, x2, theta_1, theta_2))
    print('Ex_pz_log_1_minus_D_Gz:', Ex_pz_log_1_minus_D_Gz)
    VDG = Ex_pdata_log_Dx + Ex_pz_log_1_minus_D_Gz
    print('VDG:', VDG)

<img src="img/hell.png" style="height:350px; width: auto;"/>

<img src='img/hell_2.png' style='height:350px; width: auto;' />

In [11]:
def alpha(x, theta_1, theta_2):
    # for real samples
    x1, x2 = x
    alpha_1 = (x1 * np.exp(-(theta_1 * x1 - theta_2 * x2 -2))) / (1 + np.exp(-(theta_1 * x1 - theta_2 * x2 -2)))
    alpha_2 = -(x2 * np.exp(-(theta_1 * x1 - theta_2 * x2 -2))) / (1 + np.exp(-(theta_1 * x1 - theta_2 * x2 -2)))
    return np.array([alpha_1, alpha_2])
def beta(x, theta_1, theta_2):
    # for fake samples
    x1, x2 = x
    beta_1 = -(x1 * np.exp(-(theta_1 * x1 - theta_2 * x2 -2))) / (np.exp(-(theta_1 * x1 - theta_2 * x2 -2)) * (np.exp(-(theta_1 * x1 - theta_2 * x2 -2))+1))
    beta_2 = (x2 * np.exp(-(theta_1 * x1 - theta_2 * x2 -2))) / (np.exp(-(theta_1 * x1 - theta_2 * x2 -2)) * (np.exp(-(theta_1 * x1 - theta_2 * x2 -2))+1))
    return np.array([beta_1, beta_2])

In [22]:
def minibatch(X_real, X_fake, theta_1, theta_2, k=1, lr=0.01):
    for iteration in range(k):
        print('--------------------')
        print('ITERATION', iteration)
        gradient = 0
        m = len(X_real)
        for i in range(m):
            real = X_real[i]
            fake = X_fake[i]
            alpha_x = alpha(real, theta_1, theta_2)
            beta_x = beta(fake, theta_1, theta_2)
            print('alpha:', alpha_x)
            print('beta:', beta_x)
            gradient += (1/m * (alpha_x + beta_x))
        print('gradient:', gradient) 
        # update
        new_thetas = np.array([theta_1, theta_2]) + lr * gradient
        print('new thetas', new_thetas)
        theta_1, theta_2 = new_thetas

## Examples

In [2]:
X_real = np.array([
    [1, 2],
    [3, 4]
])
X_fake = np.array([
    [5, 6],
    [7, 8]
])
theta_1 = 0.1
theta_2 = 0.2

In [5]:
compute_VDG(X_real, X_fake, theta_1, theta_2)

Proba X_real: 0.5
Ex_pdata_log_Dx: -2.4872175994452563
Ex_pz_log_1_minus_D_Gz: -0.05930316899727689
VDG: -2.5465207684425333


In [23]:
minibatch(X_real, X_fake, theta_1, theta_2, k=1, lr=0.02)

--------------------
ITERATION 0
alpha: [ 0.90887704 -1.81775408]
beta: [-0.31486678  0.37784014]
alpha: [ 2.77242546 -3.69656728]
beta: [-0.36507494  0.4172285 ]
gradient: [ 1.50068039 -2.35962636]
new thetas [0.13001361 0.15280747]
